# Data Cleaning

## Importing Libraries

In [128]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

## Cleaning Retraction Data

In [65]:
retract = pd.read_csv('./total_retraction_data.csv', index_col=False)

In [66]:
retract.head()

,Unnamed: 0,id,doi,language,year,month,day,volume,issue,journal,title,page,text,abstract,keywords,publisher,doi_check
0,0,28667474,10.1208/s12249-017-0838-6,eng,2017.0,6.0,30.0,19.0,6.0,AAPS PharmSciTech,RETRACTED ARTICLE: Development and In Vitro-In...,2750,NaN,NaN,[],NaN,10.1208/s12249-017-0838-6
1,1,27511111,10.1208/s12249-016-0596-x,eng,2016.0,8.0,10.0,18.0,5.0,AAPS PharmSciTech,Study of the Transformations of Micro/Nano-cry...,1428-1437,‘Polymorphism’ generally referred as the abili...,This study elucidates the physical properties ...,"['monoclinic', 'nano-sized crystals', 'orthorh...",Springer International Publishing,10.1208/s12249-016-0596-x
2,2,23835739,10.1208/s12249-013-0001-y,eng,2013.0,7.0,9.0,14.0,3.0,AAPS PharmSciTech,Meloxicam taste-masked oral disintegrating tab...,1118-28,NaN,The purpose of this study was to develop taste...,"['cyclodextrin', 'ion exchange resins', 'oral ...",Springer US,10.1208/s12249-013-0001-y
3,3,23800858,10.1208/s12249-013-9993-6,eng,2013.0,6.0,26.0,14.0,3.0,AAPS PharmSciTech,Design and formulation technique of a novel dr...,1045-54,NaN,"Azithromycin, an important member of the azali...","['antibacterial activity', 'azithromycin', 'ci...",Springer US,10.1208/s12249-013-9993-6
4,4,18446488,10.1208/s12249-008-9044-x,eng,2008.0,2.0,14.0,9.0,1.0,AAPS PharmSciTech,"The influence of sodium hyaluronate, L-leucine...",243-9,NaN,The purpose of this research was to evaluate t...,"['adsorption', 'betamethasone-17-valerate', 'h...",Springer-Verlag,10.1208/s12249-008-9044-x


In [67]:
retract = retract.drop(columns=['Unnamed: 0', 'doi_check'])

In [68]:
print(retract['abstract'].isnull().sum())
print(retract['text'].isnull().sum())

113
553


In [69]:
retract = retract.dropna(axis=0, subset=['text'])
print(retract.shape)
retract.head()

(1553, 15)


,id,doi,language,year,month,day,volume,issue,journal,title,page,text,abstract,keywords,publisher
1,27511111,10.1208/s12249-016-0596-x,eng,2016.0,8.0,10.0,18.0,5.0,AAPS PharmSciTech,Study of the Transformations of Micro/Nano-cry...,1428-1437,‘Polymorphism’ generally referred as the abili...,This study elucidates the physical properties ...,"['monoclinic', 'nano-sized crystals', 'orthorh...",Springer International Publishing
5,31263767,10.1021/acscentsci.9b00224,eng,2019.0,5.0,9.0,5.0,6.0,ACS central science,Targeted Protein Internalization and Degradati...,1079-1084,Traditional\ndrug development efforts are focu...,Targeted,[],American Chemical Society
7,31458862,10.1021/acsomega.8b00488,eng,2018.0,6.0,27.0,3.0,6.0,ACS omega,Regulating the Microstructure of Intumescent F...,6962-6970,Intumescent flame retardants\nare now being us...,A compatibilizer,[],American Chemical Society
8,31458855,10.1021/acsomega.8b00153,eng,2018.0,6.0,25.0,3.0,6.0,ACS omega,Solid-to-Solid Crystallization of Organic Thin...,6874-6879,Crystal growth process is basic and essential ...,The solid-to-solid crystallization processes o...,[],American Chemical Society
10,21837091,10.1107/S1600536811022574,eng,2011.0,6.0,18.0,67.0,NaN,"Acta crystallographica. Section E, Structure r...",Oxonium picrate.,o1694,For general background to organic salts of pic...,"The title compound, H3O+·C6H2N3O7",[],International Union of Crystallography


In [70]:
retract['retraction_binary'] = 1
print(retract['retraction_binary'].value_counts())
retract.head()

1    1553
Name: retraction_binary, dtype: int64


,id,doi,language,year,month,day,volume,issue,journal,title,page,text,abstract,keywords,publisher,retraction_binary
1,27511111,10.1208/s12249-016-0596-x,eng,2016.0,8.0,10.0,18.0,5.0,AAPS PharmSciTech,Study of the Transformations of Micro/Nano-cry...,1428-1437,‘Polymorphism’ generally referred as the abili...,This study elucidates the physical properties ...,"['monoclinic', 'nano-sized crystals', 'orthorh...",Springer International Publishing,1
5,31263767,10.1021/acscentsci.9b00224,eng,2019.0,5.0,9.0,5.0,6.0,ACS central science,Targeted Protein Internalization and Degradati...,1079-1084,Traditional\ndrug development efforts are focu...,Targeted,[],American Chemical Society,1
7,31458862,10.1021/acsomega.8b00488,eng,2018.0,6.0,27.0,3.0,6.0,ACS omega,Regulating the Microstructure of Intumescent F...,6962-6970,Intumescent flame retardants\nare now being us...,A compatibilizer,[],American Chemical Society,1
8,31458855,10.1021/acsomega.8b00153,eng,2018.0,6.0,25.0,3.0,6.0,ACS omega,Solid-to-Solid Crystallization of Organic Thin...,6874-6879,Crystal growth process is basic and essential ...,The solid-to-solid crystallization processes o...,[],American Chemical Society,1
10,21837091,10.1107/S1600536811022574,eng,2011.0,6.0,18.0,67.0,NaN,"Acta crystallographica. Section E, Structure r...",Oxonium picrate.,o1694,For general background to organic salts of pic...,"The title compound, H3O+·C6H2N3O7",[],International Union of Crystallography,1


In [71]:
retract.to_csv('./no_null_text_retraction_data.csv')
retract = pd.read_csv('./no_null_text_retraction_data.csv')
retract = retract.drop(columns=['Unnamed: 0'])

In [72]:
retract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553 entries, 0 to 1552
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1553 non-null   int64  
 1   doi                1553 non-null   object 
 2   language           1553 non-null   object 
 3   year               1315 non-null   float64
 4   month              1315 non-null   float64
 5   day                1315 non-null   float64
 6   volume             1518 non-null   float64
 7   issue              1043 non-null   float64
 8   journal            1553 non-null   object 
 9   title              1553 non-null   object 
 10  page               1487 non-null   object 
 11  text               1553 non-null   object 
 12  abstract           1533 non-null   object 
 13  keywords           1553 non-null   object 
 14  publisher          1549 non-null   object 
 15  retraction_binary  1553 non-null   int64  
dtypes: float64(5), int64(2),

In [ ]:
retract['text'][0]
#has large strings of formatting characters, etc. 

In [75]:
#https://stackoverflow.com/questions/40950791/remove-quotes-from-string-in-python

ls_keywords_list = []
count = 0
for i in retract['keywords']:
    keywords_list = []
    if i == []:
        ls_keywords_list.append([])
    else:
        for j in i.split():
            keywords_list.append(j.replace("'",'').replace('[','').replace(',','').replace(']','').replace('(','').replace(')','').replace('\\n', '').replace('\\n','').lower())
        if keywords_list == ['']:
            ls_keywords_list.append([])
        else:
            ls_keywords_list.append(keywords_list)

retract['unpacked_keywords'] = ls_keywords_list

In [ ]:
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

clean_text = []
clean_text_lem = []

for i in range(0, len(retract['text'])):
    ls_words = []
    ls_lem = []
    for j in tokenizer.tokenize(retract['text'][i]):
        try:
            int(j)
        except:
            if len(j) < 45:
                ls_words.append(j)
                ls_lem.append(lemmatizer.lemmatize(j))
            else:
                pass
        
    clean_text.append(' '.join(ls_words))
    clean_text_lem.append(' '.join(ls_lem))
    
    print(i)

In [153]:
print(len(clean_text[1]))
print(len(clean_text_lem[1]))

14994
14855


In [ ]:
retract['clean_text'] = clean_text
retract['clean_text_lem'] = clean_text_lem
retract.head()

In [157]:
retract.head()

,id,doi,language,year,month,day,volume,issue,journal,title,page,text,abstract,keywords,publisher,retraction_binary,unpacked_keywords,clean_text,clean_text_lem
0,27511111,10.1208/s12249-016-0596-x,eng,2016.0,8.0,10.0,18.0,5.0,AAPS PharmSciTech,Study of the Transformations of Micro/Nano-cry...,1428-1437,‘Polymorphism’ generally referred as the abili...,This study elucidates the physical properties ...,"['monoclinic', 'nano-sized crystals', 'orthorh...",Springer International Publishing,1,"[monoclinic, nano-sized, crystals, orthorhombi...",Polymorphism generally referred as the ability...,Polymorphism generally referred a the ability ...
1,31263767,10.1021/acscentsci.9b00224,eng,2019.0,5.0,9.0,5.0,6.0,ACS central science,Targeted Protein Internalization and Degradati...,1079-1084,Traditional\ndrug development efforts are focu...,Targeted,[],American Chemical Society,1,[],Traditional drug development efforts are focus...,Traditional drug development effort are focuse...
2,31458862,10.1021/acsomega.8b00488,eng,2018.0,6.0,27.0,3.0,6.0,ACS omega,Regulating the Microstructure of Intumescent F...,6962-6970,Intumescent flame retardants\nare now being us...,A compatibilizer,[],American Chemical Society,1,[],Intumescent flame retardants are now being use...,Intumescent flame retardant are now being used...
3,31458855,10.1021/acsomega.8b00153,eng,2018.0,6.0,25.0,3.0,6.0,ACS omega,Solid-to-Solid Crystallization of Organic Thin...,6874-6879,Crystal growth process is basic and essential ...,The solid-to-solid crystallization processes o...,[],American Chemical Society,1,[],Crystal growth process is basic and essential ...,Crystal growth process is basic and essential ...
4,21837091,10.1107/S1600536811022574,eng,2011.0,6.0,18.0,67.0,NaN,"Acta crystallographica. Section E, Structure r...",Oxonium picrate.,o1694,For general background to organic salts of pic...,"The title compound, H3O+·C6H2N3O7",[],International Union of Crystallography,1,[],For general background to organic salts of pic...,For general background to organic salt of picr...


## Cleaning No Retraction Data

In [76]:
text = pd.read_csv('./text__no_retraction__0_4543.csv')
abstract = pd.read_csv('./abstract__no_retraction__0_4543.csv')
keywords = pd.read_csv('./keywords__no_retraction__0_4543.csv')
publisher = pd.read_csv('./publisher__no_retraction__0_4543.csv')
doi = pd.read_csv('./doi__no_retraction__0_4543.csv')

In [77]:
ls_text = []
for i in text['0']:
    ls_text.append(i)
print(len(ls_text))
    
ls_abstract = []
for i in abstract['0']:
    ls_abstract.append(i)
print(len(ls_abstract))
    
ls_keywords = []
for i in keywords['0']:
    ls_keywords.append(i)
print(len(ls_keywords))
    
ls_publisher = []
for i in publisher['0']:
    ls_publisher.append(i)
print(len(ls_publisher))
    
ls_doi = []
for i in doi['1']:
    ls_doi.append(i)
print(len(ls_doi))

4543
4543
4543
4543
4543


In [78]:
no_retract = pd.read_csv('./pubmed_data_no_retraction_cleaned.csv')

In [79]:
no_retract = pd.concat([no_retract, pd.Series(ls_text), pd.Series(ls_abstract), pd.Series(ls_keywords), pd.Series(ls_publisher), pd.Series(ls_doi)], axis=1)

In [80]:
no_retract = no_retract.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [81]:
no_retract = no_retract.rename(columns={'0':'id', '1':'doi', '2':'language', '3':'year', '4':'month', '5':'day', 
                      '6':'volume', '7':'issue', '8':'journal', '9':'title', '10':'page', 
                      0:'text', 1:'abstract', 2:'keywords', 3:'publisher', 4:'doi_check'})

In [82]:
no_retract.head()

,id,doi,language,year,month,day,volume,issue,journal,title,page,text,abstract,keywords,publisher,doi_check
0,33014689,10.1007/s13205-020-02432-w,eng,2020.0,9.0,19.0,10.0,10.0,3 Biotech,AuNPs/CNF-modified DNA biosensor for early and...,446,NaN,NaN,[],NaN,10.1007/s13205-020-02432-w
1,33014688,10.1007/s13205-020-02435-7,eng,2020.0,9.0,19.0,10.0,10.0,3 Biotech,NF-κB signaling induces inductive expression o...,445,NaN,NaN,[],NaN,10.1007/s13205-020-02435-7
2,33006702,10.1186/s41205-020-00083-4,eng,2020.0,10.0,2.0,6.0,1.0,3D printing in medicine,Clinical applications of custom 3D printed imp...,29,Three dimensional printing has greatly advance...,Background,"['Implants', 'Foot and ankle', 'Orthopaedic su...",Springer International Publishing,10.1186/s41205-020-00083-4
3,32997313,10.1186/s41205-020-00081-6,eng,2020.0,9.0,30.0,6.0,1.0,3D printing in medicine,3D printing in critical care: a narrative review.,28,3D printing (3DP) has gained interest in many ...,Background,"['3D printing', 'Critical care', 'Medical educ...",Springer International Publishing,10.1186/s41205-020-00081-6
4,32984538,10.4158/ACCR-2020-0076,eng,2020.0,8.0,6.0,6.0,5.0,AACE clinical case reports,CONVERSION OF HYPOTHYROIDISM TO HYPERTHYROIDIS...,e279-e281,We present a patient with long standing hypoth...,Objective:,[],American Association of Clinical Endocrinologists,10.4158/ACCR-2020-0076


In [83]:
no_retract['doi'].head(10)

0    10.1007/s13205-020-02432-w
1    10.1007/s13205-020-02435-7
2    10.1186/s41205-020-00083-4
3    10.1186/s41205-020-00081-6
4        10.4158/ACCR-2020-0076
5        10.4158/ACCR-2020-0051
6    10.1208/s12248-020-00504-6
7    10.1208/s12248-020-00481-w
8    10.1208/s12249-020-01811-z
9    10.1208/s12249-020-01809-7
Name: doi, dtype: object

In [84]:
no_retract['doi_check'].head(10)

0    10.1007/s13205-020-02432-w
1    10.1007/s13205-020-02435-7
2    10.1186/s41205-020-00083-4
3    10.1186/s41205-020-00081-6
4        10.4158/ACCR-2020-0076
5        10.4158/ACCR-2020-0051
6    10.1208/s12248-020-00504-6
7    10.1208/s12248-020-00481-w
8    10.1208/s12249-020-01811-z
9    10.1208/s12249-020-01809-7
Name: doi_check, dtype: object

In [85]:
no_retract.to_csv('./total_no_retraction_data.csv')

In [86]:
no_retract = pd.read_csv('./total_no_retraction_data.csv', index_col=False)
no_retract = no_retract.drop(columns=['Unnamed: 0', 'doi_check'])

In [87]:
print(no_retract['abstract'].isnull().sum())
print(no_retract['text'].isnull().sum())

1924
1903


In [88]:
no_retract = no_retract.dropna(axis=0, subset=['text'])
print(no_retract.shape)
no_retract.head()

(3432, 15)


,id,doi,language,year,month,day,volume,issue,journal,title,page,text,abstract,keywords,publisher
2,33006702,10.1186/s41205-020-00083-4,eng,2020.0,10.0,2.0,6.0,1.0,3D printing in medicine,Clinical applications of custom 3D printed imp...,29,Three dimensional printing has greatly advance...,Background,"['Implants', 'Foot and ankle', 'Orthopaedic su...",Springer International Publishing
3,32997313,10.1186/s41205-020-00081-6,eng,2020.0,9.0,30.0,6.0,1.0,3D printing in medicine,3D printing in critical care: a narrative review.,28,3D printing (3DP) has gained interest in many ...,Background,"['3D printing', 'Critical care', 'Medical educ...",Springer International Publishing
4,32984538,10.4158/ACCR-2020-0076,eng,2020.0,8.0,6.0,6.0,5.0,AACE clinical case reports,CONVERSION OF HYPOTHYROIDISM TO HYPERTHYROIDIS...,e279-e281,We present a patient with long standing hypoth...,Objective:,[],American Association of Clinical Endocrinologists
5,32984537,10.4158/ACCR-2020-0051,eng,2020.0,9.0,21.0,6.0,5.0,AACE clinical case reports,INCIDENTAL PULMONARY METASTASES REVEALING SUBC...,e273-e278,Here we present 2 cases of papillary thyroid m...,Objective:,[],American Association of Clinical Endocrinologists
10,32964195,10.12688/aasopenres.12999.1,eng,2020.0,8.0,4.0,3.0,NaN,AAS open research,SNPs in IL4 and IFNG show no protective associ...,35,"Human African trypanosomiasis (HAT), or sleepi...",Background: Human African trypanosomiasis (HAT...,"['Human African Trypanosomiasis', 'T.b.gambien...",F1000 Research Limited


In [89]:
no_retract['retraction_binary'] = 0
print(no_retract['retraction_binary'].value_counts())
no_retract.head()

0    3432
Name: retraction_binary, dtype: int64


,id,doi,language,year,month,day,volume,issue,journal,title,page,text,abstract,keywords,publisher,retraction_binary
2,33006702,10.1186/s41205-020-00083-4,eng,2020.0,10.0,2.0,6.0,1.0,3D printing in medicine,Clinical applications of custom 3D printed imp...,29,Three dimensional printing has greatly advance...,Background,"['Implants', 'Foot and ankle', 'Orthopaedic su...",Springer International Publishing,0
3,32997313,10.1186/s41205-020-00081-6,eng,2020.0,9.0,30.0,6.0,1.0,3D printing in medicine,3D printing in critical care: a narrative review.,28,3D printing (3DP) has gained interest in many ...,Background,"['3D printing', 'Critical care', 'Medical educ...",Springer International Publishing,0
4,32984538,10.4158/ACCR-2020-0076,eng,2020.0,8.0,6.0,6.0,5.0,AACE clinical case reports,CONVERSION OF HYPOTHYROIDISM TO HYPERTHYROIDIS...,e279-e281,We present a patient with long standing hypoth...,Objective:,[],American Association of Clinical Endocrinologists,0
5,32984537,10.4158/ACCR-2020-0051,eng,2020.0,9.0,21.0,6.0,5.0,AACE clinical case reports,INCIDENTAL PULMONARY METASTASES REVEALING SUBC...,e273-e278,Here we present 2 cases of papillary thyroid m...,Objective:,[],American Association of Clinical Endocrinologists,0
10,32964195,10.12688/aasopenres.12999.1,eng,2020.0,8.0,4.0,3.0,NaN,AAS open research,SNPs in IL4 and IFNG show no protective associ...,35,"Human African trypanosomiasis (HAT), or sleepi...",Background: Human African trypanosomiasis (HAT...,"['Human African Trypanosomiasis', 'T.b.gambien...",F1000 Research Limited,0


In [90]:
no_retract.to_csv('./no_null_text_no_retraction_data.csv')
no_retract = pd.read_csv('./no_null_text_no_retraction_data.csv')
no_retract = no_retract.drop(columns=['Unnamed: 0'])

In [91]:
no_retract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3432 entries, 0 to 3431
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 3432 non-null   int64  
 1   doi                3432 non-null   object 
 2   language           3432 non-null   object 
 3   year               3073 non-null   float64
 4   month              3073 non-null   float64
 5   day                3073 non-null   float64
 6   volume             3104 non-null   float64
 7   issue              1668 non-null   float64
 8   journal            3432 non-null   object 
 9   title              3432 non-null   object 
 10  page               3159 non-null   object 
 11  text               3432 non-null   object 
 12  abstract           3212 non-null   object 
 13  keywords           3432 non-null   object 
 14  publisher          3424 non-null   object 
 15  retraction_binary  3432 non-null   int64  
dtypes: float64(5), int64(2),

In [92]:
#https://stackoverflow.com/questions/40950791/remove-quotes-from-string-in-python

ls_keywords_list = []
count = 0
for i in no_retract['keywords']:
    keywords_list = []
    if i == []:
        ls_keywords_list.append([])
    else:
        for j in i.split():
            keywords_list.append(j.replace("'",'').replace('[','').replace(',','').replace(']','').replace('(','').replace(')','').replace('\\n', '').replace('\\n','').lower())
        if keywords_list == ['']:
            ls_keywords_list.append([])
        else:
            ls_keywords_list.append(keywords_list)

no_retract['unpacked_keywords'] = ls_keywords_list

In [ ]:
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

clean_text = []
clean_text_lem = []

for i in range(0, len(no_retract['text'])):
    ls_words = []
    ls_lem = []
    for j in tokenizer.tokenize(no_retract['text'][i]):
        try:
            int(j)
        except:
            if len(j) < 45:
                ls_words.append(j)
                ls_lem.append(lemmatizer.lemmatize(j))
            else:
                pass
        
    clean_text.append(' '.join(ls_words))
    clean_text_lem.append(' '.join(ls_lem))
    
    print(i)

print(len(no_retract['text'][1]))
print(len(clean_text[1]))
print(len(clean_text_lem[1]))

In [156]:
no_retract['clean_text'] = clean_text
no_retract['clean_text_lem'] = clean_text_lem
no_retract.head()

,id,doi,language,year,month,day,volume,issue,journal,title,page,text,abstract,keywords,publisher,retraction_binary,unpacked_keywords,clean_text,clean_text_lem
0,33006702,10.1186/s41205-020-00083-4,eng,2020.0,10.0,2.0,6.0,1.0,3D printing in medicine,Clinical applications of custom 3D printed imp...,29,Three dimensional printing has greatly advance...,Background,"['Implants', 'Foot and ankle', 'Orthopaedic su...",Springer International Publishing,0,"[implants, foot, and, ankle, orthopaedic, surg...",Three dimensional printing has greatly advance...,Three dimensional printing ha greatly advanced...
1,32997313,10.1186/s41205-020-00081-6,eng,2020.0,9.0,30.0,6.0,1.0,3D printing in medicine,3D printing in critical care: a narrative review.,28,3D printing (3DP) has gained interest in many ...,Background,"['3D printing', 'Critical care', 'Medical educ...",Springer International Publishing,0,"[3d, printing, critical, care, medical, educat...",3D printing 3DP has gained interest in many fi...,3D printing 3DP ha gained interest in many fie...
2,32984538,10.4158/ACCR-2020-0076,eng,2020.0,8.0,6.0,6.0,5.0,AACE clinical case reports,CONVERSION OF HYPOTHYROIDISM TO HYPERTHYROIDIS...,e279-e281,We present a patient with long standing hypoth...,Objective:,[],American Association of Clinical Endocrinologists,0,[],We present a patient with long standing hypoth...,We present a patient with long standing hypoth...
3,32984537,10.4158/ACCR-2020-0051,eng,2020.0,9.0,21.0,6.0,5.0,AACE clinical case reports,INCIDENTAL PULMONARY METASTASES REVEALING SUBC...,e273-e278,Here we present 2 cases of papillary thyroid m...,Objective:,[],American Association of Clinical Endocrinologists,0,[],Here we present cases of papillary thyroid mic...,Here we present case of papillary thyroid micr...
4,32964195,10.12688/aasopenres.12999.1,eng,2020.0,8.0,4.0,3.0,NaN,AAS open research,SNPs in IL4 and IFNG show no protective associ...,35,"Human African trypanosomiasis (HAT), or sleepi...",Background: Human African trypanosomiasis (HAT...,"['Human African Trypanosomiasis', 'T.b.gambien...",F1000 Research Limited,0,"[human, african, trypanosomiasis, t.b.gambiens...",Human African trypanosomiasis HAT or sleeping ...,Human African trypanosomiasis HAT or sleeping ...


## Cleaning Both Datasets

In [158]:
total = pd.concat([retract, no_retract], axis=0)
total

,id,doi,language,year,month,day,volume,issue,journal,title,page,text,abstract,keywords,publisher,retraction_binary,unpacked_keywords,clean_text,clean_text_lem
0,27511111,10.1208/s12249-016-0596-x,eng,2016.0,8.0,10.0,18.0,5.0,AAPS PharmSciTech,Study of the Transformations of Micro/Nano-cry...,1428-1437,‘Polymorphism’ generally referred as the abili...,This study elucidates the physical properties ...,"['monoclinic', 'nano-sized crystals', 'orthorh...",Springer International Publishing,1,"[monoclinic, nano-sized, crystals, orthorhombi...",Polymorphism generally referred as the ability...,Polymorphism generally referred a the ability ...
1,31263767,10.1021/acscentsci.9b00224,eng,2019.0,5.0,9.0,5.0,6.0,ACS central science,Targeted Protein Internalization and Degradati...,1079-1084,Traditional\ndrug development efforts are focu...,Targeted,[],American Chemical Society,1,[],Traditional drug development efforts are focus...,Traditional drug development effort are focuse...
2,31458862,10.1021/acsomega.8b00488,eng,2018.0,6.0,27.0,3.0,6.0,ACS omega,Regulating the Microstructure of Intumescent F...,6962-6970,Intumescent flame retardants\nare now being us...,A compatibilizer,[],American Chemical Society,1,[],Intumescent flame retardants are now being use...,Intumescent flame retardant are now being used...
3,31458855,10.1021/acsomega.8b00153,eng,2018.0,6.0,25.0,3.0,6.0,ACS omega,Solid-to-Solid Crystallization of Organic Thin...,6874-6879,Crystal growth process is basic and essential ...,The solid-to-solid crystallization processes o...,[],American Chemical Society,1,[],Crystal growth process is basic and essential ...,Crystal growth process is basic and essential ...
4,21837091,10.1107/S1600536811022574,eng,2011.0,6.0,18.0,67.0,NaN,"Acta crystallographica. Section E, Structure r...",Oxonium picrate.,o1694,For general background to organic salts of pic...,"The title compound, H3O+·C6H2N3O7",[],International Union of Crystallography,1,[],For general background to organic salts of pic...,For general background to organic salt of picr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3427,30510799,10.1155/2018/3793592,eng,2018.0,10.0,29.0,2018.0,NaN,Plastic surgery international,Surgical Correction of Unicoronal Craniosynost...,3793592,Craniosynostosis is the premature fusion of o...,Background,[],Hindawi,0,[],Craniosynostosis is the premature fusion of on...,Craniosynostosis is the premature fusion of on...
3428,30155297,10.1155/2018/6917454,eng,2018.0,8.0,1.0,2018.0,NaN,Plastic surgery international,"Response to: Comment on ""Evaluation of Complic...",6917454,DIZG (German Institute for Cell and Tissue Rep...,NaN,[],Hindawi,0,[],DIZG German Institute for Cell and Tissue Repl...,DIZG German Institute for Cell and Tissue Repl...
3429,32934974,10.1515/pp-2019-0027,eng,2020.0,2.0,26.0,5.0,1.0,Pleura and peritoneum,Two vs. three weeks of treatment with amoxicil...,20190027,The optimal duration of antibiotic treatment f...,Abstract,"['amoxicillin-clavulanate', 'community-acquire...",De Gruyter,0,"[amoxicillin-clavulanate, community-acquired, ...",The optimal duration of antibiotic treatment f...,The optimal duration of antibiotic treatment f...
3430,31032144,ecurrents.outbreaks.7a6c64436a3085ebba37e5329b...,eng,2018.0,9.0,28.0,10.0,NaN,PLoS currents,Measuring Mosquito-borne Viral Suitability in ...,NaN,"In South East Asia, mosquito-borne viruses (MB...",Introduction:,[],Public Library of Science,0,[],In South East Asia mosquito borne viruses MBVs...,In South East Asia mosquito borne virus MBVs h...


In [159]:
total = total.drop(columns=['id', 'language'])

In [160]:
total['page'].value_counts()

3         23
4         22
1         21
26        18
16        17
          ..
100276     1
540516     1
e23594     1
e84738     1
40-41      1
Name: page, Length: 3740, dtype: int64

In [161]:
#Drop page column as the same information can be basically derived from the character length
#as well as inconsistent formatting
total = total.drop(columns=['page'])

In [162]:
#Change year, month, day, volume, issue into strings for consistency
#See if need to dummy after EDA
total['year'] = total['year'].astype(str)
total['month'] = total['month'].astype(str)
total['day'] = total['day'].astype(str)
total['volume'] = total['volume'].astype(str)
total['issue'] = total['issue'].astype(str)

In [163]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4985 entries, 0 to 3431
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   doi                4985 non-null   object
 1   year               4985 non-null   object
 2   month              4985 non-null   object
 3   day                4985 non-null   object
 4   volume             4985 non-null   object
 5   issue              4985 non-null   object
 6   journal            4985 non-null   object
 7   title              4985 non-null   object
 8   text               4985 non-null   object
 9   abstract           4745 non-null   object
 10  keywords           4985 non-null   object
 11  publisher          4973 non-null   object
 12  retraction_binary  4985 non-null   int64 
 13  unpacked_keywords  4985 non-null   object
 14  clean_text         4985 non-null   object
 15  clean_text_lem     4985 non-null   object
dtypes: int64(1), object(15)
memory usage: 662.

In [165]:
#for indexing purposes
total.to_csv('./total_data.csv')
total = pd.read_csv('./total_data.csv')

In [166]:
#need to fix duplicates that exist
total['doi'].value_counts().head(19)

10.1074/jbc.M111.329078           4
10.1155/2012/236409               2
10.1074/jbc.M111.275073           2
10.1074/jbc.M709854200            2
10.1038/cr.2011.194               2
10.1186/s12711-020-00577-z        2
10.1074/jbc.M808084200            2
10.1038/cdd.2010.114              2
10.1186/s12711-020-00575-1        2
10.1074/jbc.M112.387738           2
10.1186/s12863-020-00920-9        2
10.1186/s13053-020-00152-z        2
10.1155/2014/123918               2
10.1074/jbc.M110.175802           2
10.3389/fnins.2018.00529          2
10.1523/JNEUROSCI.0372-13.2013    2
10.1200/JCO.2017.74.7824          2
10.3390/nano4020203               2
10.3114/fuse.2020.06.06           1
Name: doi, dtype: int64

In [167]:
print(total.loc[total['doi']=='10.1074/jbc.M111.329078'].index)
print(total.loc[total['doi']=='10.1186/s13053-020-00152-z'].index)
print(total.loc[total['doi']=='10.1074/jbc.M112.387738'].index)
print(total.loc[total['doi']=='10.1038/cr.2011.194'].index)
print(total.loc[total['doi']=='10.1074/jbc.M808084200'].index)
print(total.loc[total['doi']=='10.3389/fnins.2018.00529'].index)
print(total.loc[total['doi']=='10.1074/jbc.M111.275073'].index)
print(total.loc[total['doi']=='10.1186/s12711-020-00577-z'].index)
print(total.loc[total['doi']=='10.1200/JCO.2017.74.7824'].index)
print(total.loc[total['doi']=='10.1155/2014/123918'].index)
print(total.loc[total['doi']=='10.1074/jbc.M709854200'].index)
print(total.loc[total['doi']=='10.1038/cdd.2010.114'].index)
print(total.loc[total['doi']=='10.1155/2012/236409'].index)
print(total.loc[total['doi']=='10.1186/s12863-020-00920-9'].index)
print(total.loc[total['doi']=='10.1074/jbc.M110.175802'].index)
print(total.loc[total['doi']=='10.3390/nano4020203'].index)
print(total.loc[total['doi']=='10.1523/JNEUROSCI.0372-13.2013'].index)
print(total.loc[total['doi']=='10.1186/s12711-020-00575-1'].index)

Int64Index([56, 60, 447, 676], dtype='int64')
Int64Index([1672, 3251], dtype='int64')
Int64Index([662, 1043], dtype='int64')
Int64Index([265, 448], dtype='int64')
Int64Index([752, 1057], dtype='int64')
Int64Index([417, 1019], dtype='int64')
Int64Index([682, 1048], dtype='int64')
Int64Index([3092, 3094], dtype='int64')
Int64Index([57, 1219], dtype='int64')
Int64Index([588, 3692], dtype='int64')
Int64Index([769, 1059], dtype='int64')
Int64Index([248, 1052], dtype='int64')
Int64Index([520, 997], dtype='int64')
Int64Index([2076, 3098], dtype='int64')
Int64Index([714, 1053], dtype='int64')
Int64Index([1098, 1237], dtype='int64')
Int64Index([893, 1042], dtype='int64')
Int64Index([3093, 3095], dtype='int64')


In [168]:
total = total.drop(total.index[[60, 447, 676, 3251, 1043, 448, 1057, 1019, 1048, 3094, 1219, 3692, 1059,
                   1052, 997, 3098, 1053, 1237, 1042, 3095]])

In [169]:
total['doi'].value_counts().head()

10.1186/s12943-020-01262-x    1
10.1177/1179552218819492      1
10.1105/tpc.111.088815        1
10.1186/s12981-020-00312-0    1
10.2147/BCTT.S17892           1
Name: doi, dtype: int64

In [170]:
total.to_csv('./total_data_no_duplicates.csv')
total = pd.read_csv('./total_data_no_duplicates.csv')

In [171]:
total['clean_text'][950:955]

950    For the chemotherapeutic activity of pyrimidin...
951    Urinary tract infection UTI is a bacterial inf...
952    The Bible descrbies the case of a woman with h...
953                                                  NaN
954    Human betaherpesviruses 6A and 6B HHV 6A and H...
Name: clean_text, dtype: object

In [172]:
total = total.drop(953)
total['clean_text'][950:955]

950    For the chemotherapeutic activity of pyrimidin...
951    Urinary tract infection UTI is a bacterial inf...
952    The Bible descrbies the case of a woman with h...
954    Human betaherpesviruses 6A and 6B HHV 6A and H...
955    A cesarean section is the most frequently perf...
Name: clean_text, dtype: object

In [175]:
total['retraction_binary'][:1538]

0       1
1       1
2       1
3       1
4       1
       ..
1534    1
1535    1
1536    1
1537    1
1538    0
Name: retraction_binary, Length: 1538, dtype: int64

In [176]:
total.to_csv('./total_data_cleaned.csv')
total[1537:].to_csv('./no_retraction_data_cleaned.csv')
total[:1537].to_csv('./retraction_data_cleaned.csv')